In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Embedding, GRU, Dense
from keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [ ]:
# Load your dataset from the CSV file
data = pd.read_csv('/kaggle/input/cleaneddataset/cleaned_data_v2.csv')

In [ ]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(data['essay'], data['scores'], test_size=0.2, random_state=42)

In [ ]:
# Preprocess the text data using GloVe embeddings
embedding_dim = 100
vocab_size = 10000

In [ ]:
# Load the pre-trained GloVe embedding weights
glove_file = '/kaggle/input/glove100d/glove.6B.100d.txt'
embedding_index = {}
with open(glove_file, 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs

In [ ]:
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenize the text data
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(X_train)

# Convert text to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences to the same length
max_length = max([len(seq) for seq in X_train_seq])
X_train_padded = pad_sequences(X_train_seq, maxlen=max_length)
X_test_padded = pad_sequences(X_test_seq, maxlen=max_length)

# Create the embedding matrix
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i < vocab_size:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector


In [ ]:
# Create the model
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_length, weights=[embedding_matrix], trainable=False))
model.add(GRU(100, return_sequences=True))
model.add(GRU(64))
model.add(Dense(1))

In [ ]:
import tensorflow as tf

def r2(y_true, y_pred):
    SS_res = tf.reduce_sum(tf.square(y_true - y_pred))
    SS_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true)))
    return (1 - SS_res / (SS_tot + tf.keras.backend.epsilon()))


In [ ]:
model.compile(loss='mean_squared_error', optimizer=RMSprop(lr=0.0006), metrics=['mse', r2])


In [ ]:
# Set up early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [ ]:
# Train the model
history = model.fit(X_train_padded, y_train, batch_size=128, epochs=100, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/100
45/45 [==============================] - 4s 92ms/step - loss: 0.2591 - mse: 0.2591 - r2: 0.8248 - val_loss: 0.2942 - val_mse: 0.2942 - val_r2: 0.7967
Epoch 2/100
45/45 [==============================] - 4s 90ms/step - loss: 0.2509 - mse: 0.2509 - r2: 0.8307 - val_loss: 0.2924 - val_mse: 0.2924 - val_r2: 0.7995
Epoch 3/100
45/45 [==============================] - 4s 92ms/step - loss: 0.2538 - mse: 0.2538 - r2: 0.8287 - val_loss: 0.2651 - val_mse: 0.2651 - val_r2: 0.8184
Epoch 4/100
45/45 [==============================] - 4s 96ms/step - loss: 0.2476 - mse: 0.2476 - r2: 0.8343 - val_loss: 0.2801 - val_mse: 0.2801 - val_r2: 0.8082
Epoch 5/100
45/45 [==============================] - 4s 86ms/step - loss: 0.2418 - mse: 0.2418 - r2: 0.8370 - val_loss: 0.2645 - val_mse: 0.2645 - val_r2: 0.8186
Epoch 6/100
45/45 [==============================] - 4s 91ms/step - loss: 0.2409 - mse: 0.2409 - r2: 0.8377 - val_loss: 0.2625 - val_mse: 0.2625 - val_r2: 0.8199
Epoch 7/100
45/45 [=========

In [ ]:
# Evaluate the model
y_pred = model.predict(X_test_padded)
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
print("R-squared:", r2_score(y_test, y_pred))

57/57 [==============================] - 2s 26ms/step
Mean Squared Error: 0.2794251532510468
R-squared: 0.8220081990143536
